In [9]:
import torch
from models import upvote_predictor, skipgram
import pandas as pd
import os
from util import artifacts, cache
import tokenization
import embeddings
import numpy as np

import importlib
importlib.reload(artifacts)

<module 'util.artifacts' from '/Users/cameron/Documents/mlx/hackernews/util/artifacts.py'>

In [7]:
state_dict = artifacts.load_artifact('predictor-state')
vocab = artifacts.load_artifact('vocab')
embed_scaler = artifacts.load_artifact('predictor-embed-scaler')
karma_scaler = artifacts.load_artifact('predictor-karma-scaler')

model = upvote_predictor.Model(skipgram.EMBEDDING_DIM)

model.load_state_dict(state_dict)
model.eval()

KeyError: 'predictor-state'

In [ ]:
items_table = "hacker_news.items"

hn_test_posts = cache.query("hn_posts_for_test", f"""SELECT
    title,
    karma,
    score
    FROM {items_table}
    INNER JOIN hacker_news.users u ON {items_table}.by = u.id
    WHERE type = 'story' AND title IS NOT null
    OFFSET 1000000
    LIMIT 10
""")

hn_test_posts.dropna(inplace=True)

hn_test_posts['embeddings'] = hn_test_posts['title'].apply(embeddings.get_embeddings_for_title)

hn_test_posts.dropna()

title_embeddings = np.vstack(np.array(hn_test_posts['embeddings'].values))
title_embeddings = embed_scaler.transform(title_embeddings)

karma = hn_test_posts['karma'].values.reshape(-1, 1)
karma = karma_scaler.transform(karma)

def eval_test_row(row):
    embeddings = row['embeddings']
    karma = row['karma']
    return model(embeddings, karma)

hn_test_posts['predicted_score'] = hn_test_posts.apply(eval_test_row)
hn_test_posts

wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=19 sec). Create a new API with an integer timeout larger than 19, e.g., `api = wandb.Api(timeout=29)` to increase the graphql timeout.
wandb:   1 of 1 files downloaded.  


KeyError: 'embed-scaler'